## Corpus de español: 

* AnCora | Github: https://github.com/UniversalDependencies/UD_Spanish-AnCora

* usamos el conllu parser para leer el corpus: https://pypi.org/project/conllu/

* Etiquetas Universal POS (Documentación): https://universaldependencies.org/u/pos/



In [1]:
#@title dependencias previas
!pip install conllu
!git clone https://github.com/UniversalDependencies/UD_Spanish-AnCora.git

Cloning into 'UD_Spanish-AnCora'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 518 (delta 10), reused 18 (delta 7), pack-reused 495
Receiving objects: 100% (518/518), 115.95 MiB | 28.67 MiB/s, done.
Resolving deltas: 100% (357/357), done.


In [2]:
#@title leyendo el corpus AnCora
from conllu import parse_incr 


In [3]:
wordList = []
data_file = open("UD_Spanish-AnCora/es_ancora-ud-dev.conllu", "r", encoding="utf-8")
for tokenlist in parse_incr(data_file):
    print(tokenlist.serialize())

Se truncaron las últimas líneas 5000 del resultado de transmisión.
12	lo	él	PRON	PRON	Case=Acc|Gender=Masc|Number=Sing|Person=3|PrepCase=Npr|PronType=Prs	13	obj	_	_
13	hace	hacer	VERB	VERB	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin	4	acl	_	_
14	con	con	ADP	ADP	AdpType=Prep	15	case	_	_
15	10	10	NUM	NUM	NumForm=Digit|NumType=Card	13	obl	_	_
16	y	y	CCONJ	CCONJ	_	18	cc	_	_
17	ahora	ahora	ADV	ADV	_	18	advmod	_	_
18	esto	este	PRON	PRON	Number=Sing|PronType=Dem	4	conj	_	_
19	"	"	PUNCT	PUNCT	PunctType=Quot	4	punct	_	SpaceAfter=No
20	,	,	PUNCT	PUNCT	PunctType=Comm	4	punct	_	_
21	añade	añadir	VERB	VERB	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin	0	root	_	_
22	el	el	DET	DET	Definite=Def|Gender=Masc|Number=Sing|PronType=Art	23	det	_	_
23	técnico	técnico	NOUN	NOUN	Gender=Masc|Number=Sing	21	nsubj	_	SpaceAfter=No
24	,	,	PUNCT	PUNCT	PunctType=Comm	26	punct	_	_
25	un	uno	DET	DET	Definite=Ind|Gender=Masc|Number=Sing|PronType=Art	26	det	_	_
26	fotógrafo	fotógrafo	NOUN	NOUN	Gender=Ma

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [4]:
#@title Estructura de los tokens etiquetados del corpus
tokenlist[1]

{'deprel': 'nsubj',
 'deps': None,
 'feats': {'Gender': 'Masc', 'Number': 'Sing'},
 'form': 'cierto',
 'head': 3,
 'id': 2,
 'lemma': 'cierto',
 'misc': None,
 'upos': 'ADJ',
 'xpos': 'ADJ'}

In [5]:
tokenlist[1]['form']+'|'+tokenlist[1]['upos']

'cierto|ADJ'

## Entrenamiento del modelo - Calculo de conteos:

* tags (tags) `tagCountDict`: $C(tag)$
* emisiones (word|tag) `emissionProbDict`: $C(word|tag)$
* transiciones (tag|prevtag) `transitionDict`: $C(tag|prevtag)$

In [6]:
tagCountDict = {} 
emissionDict = {}
transitionDict = {}

tagtype = 'upos'
data_file = open("UD_Spanish-AnCora/es_ancora-ud-dev.conllu", "r", encoding="utf-8")


In [7]:
# Calculando conteos (pre-probabilidades)
for tokenlist in parse_incr(data_file):
  prevtag = None
  for token in tokenlist:

    # C(tag)
    tag = token[tagtype]
    if tag in tagCountDict.keys():
      tagCountDict[tag] += 1
    else:
      tagCountDict[tag] = 1

    # C(word|tag) -> probabilidades emision
    wordtag = token['form'].lower()+'|'+token[tagtype] # (word|tag)
    if wordtag in emissionDict.keys():
      emissionDict[wordtag] = emissionDict[wordtag] + 1
    else:
      emissionDict[wordtag] = 1

    #  C(tag|tag_previo) -> probabilidades transición
    if prevtag is None:
      prevtag = tag
      continue
    transitiontags = tag+'|'+prevtag
    if transitiontags in transitionDict.keys():
      transitionDict[transitiontags] = transitionDict[transitiontags] + 1
    else:
      transitionDict[transitiontags] = 1
    prevtag = tag
    


In [8]:
transitionDict
#emissionDict
#tagCountDict

{'ADJ|ADJ': 107,
 'ADJ|ADP': 177,
 'ADJ|ADV': 226,
 'ADJ|AUX': 157,
 'ADJ|CCONJ': 123,
 'ADJ|DET': 631,
 'ADJ|NOUN': 1635,
 'ADJ|NUM': 30,
 'ADJ|PRON': 49,
 'ADJ|PROPN': 62,
 'ADJ|PUNCT': 201,
 'ADJ|SCONJ': 10,
 'ADJ|SYM': 1,
 'ADJ|VERB': 113,
 'ADP|ADJ': 989,
 'ADP|ADP': 24,
 'ADP|ADV': 310,
 'ADP|AUX': 78,
 'ADP|CCONJ': 144,
 'ADP|DET': 30,
 'ADP|INTJ': 1,
 'ADP|NOUN': 3676,
 'ADP|NUM': 176,
 'ADP|PRON': 230,
 'ADP|PROPN': 579,
 'ADP|PUNCT': 695,
 'ADP|SCONJ': 78,
 'ADP|SYM': 25,
 'ADP|VERB': 1219,
 'ADV|ADJ': 61,
 'ADV|ADP': 122,
 'ADV|ADV': 94,
 'ADV|AUX': 69,
 'ADV|CCONJ': 103,
 'ADV|DET': 35,
 'ADV|NOUN': 200,
 'ADV|NUM': 4,
 'ADV|PART': 6,
 'ADV|PRON': 126,
 'ADV|PROPN': 60,
 'ADV|PUNCT': 240,
 'ADV|SCONJ': 87,
 'ADV|SYM': 1,
 'ADV|VERB': 358,
 'AUX|ADJ': 54,
 'AUX|ADP': 45,
 'AUX|ADV': 162,
 'AUX|AUX': 66,
 'AUX|CCONJ': 28,
 'AUX|NOUN': 196,
 'AUX|NUM': 6,
 'AUX|PRON': 314,
 'AUX|PROPN': 108,
 'AUX|PUNCT': 156,
 'AUX|SCONJ': 95,
 'AUX|VERB': 31,
 'AUX|_': 2,
 'CCONJ|ADJ': 237,


## Entrenamiento del modelo - calculo de probabilidades
* probabilidades de transición:
$$P(tag|prevtag) = \frac{C(prevtag, tag)}{C(prevtag)}$$

* probabilidades de emisión:
 $$P(word|tag) = \frac{C(word|tag)}{C(tag)}$$

In [9]:
transitionProbDict = {} # matriz A
emissionProbDict = {} # matriz B

# transition Probabilities 
for key in transitionDict.keys():
  tag, prevtag = key.split('|')
  if tagCountDict[prevtag]>0:
    transitionProbDict[key] = transitionDict[key]/(tagCountDict[prevtag])
  else:
    print(key)



In [10]:
# emission Probabilities 
for key in emissionDict.keys():
  word, tag = key.split('|')
  if emissionDict[key]>0:
    emissionProbDict[key] = emissionDict[key]/tagCountDict[tag]
  else:
    print(key)



In [11]:
#transitionProbDict
#transitionProbDict['ADJ|ADJ']
#emissionProbDict

In [12]:
import numpy as np

In [13]:
np.save('transitionHMM.npy', transitionProbDict)
np.save('emissionHMM.npy', emissionProbDict)
transitionProbdict = np.load('transitionHMM.npy', allow_pickle='TRUE').item()


In [14]:
transitionProbDict['ADJ|ADJ']

0.030225988700564973